In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import pandas as pd
import numpy as np


# Read the file for the model
model_df = pd.read_csv(r'C:\Users\pola_\OneDrive\Documents\Cesar\Data-bootcamp-UoT\Lessons\week 23\final project\Final_Project_Machine_Learning\Resources\clean_sampled_US_Accidents_for_model.csv')

model_df

,ID,Severity,Start_Time,Start_Lat,Start_Lng,Distance(mi),County,State,Temperature(F),Humidity(%),...,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Astronomical_Twilight,Year,Month,DayOfWeek,Time
0,A-6796407,2,2020-11-03 00:54:07,37.447849,-77.568042,0.237,Chesterfield,VA,61.659699,64.838302,...,False,False,False,False,False,Night,2020,11,1,00:54:07
1,A-1381523,3,2020-07-08 13:09:55,32.776855,-97.212555,0.000,Tarrant,TX,89.000000,67.000000,...,False,False,False,False,False,Day,2020,7,2,13:09:55
2,A-4503744,2,2022-05-18 17:18:44,30.008792,-90.024953,0.250,Orleans,LA,88.000000,57.000000,...,False,False,False,False,False,Day,2022,5,2,17:18:44
3,A-1179379,2,2021-01-26 05:01:29,39.586254,-77.819359,0.000,Washington,MD,31.000000,89.000000,...,False,False,False,False,False,Night,2021,1,1,05:01:29
4,A-122310,2,2017-01-12 14:03:42,25.627335,-80.379433,0.010,Miami-Dade,FL,79.000000,54.000000,...,False,False,False,True,False,Day,2017,1,3,14:03:42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535096,A-6487662,2,2021-01-21 00:16:00,28.542705,-81.281050,0.722,Orange,FL,56.000000,49.000000,...,False,False,False,False,False,Night,2021,1,3,00:16:00
3535097,A-1640736,2,2020-02-10 13:04:26,34.023918,-118.191162,0.000,Los Angeles,CA,72.000000,22.000000,...,False,False,False,True,False,Day,2020,2,0,13:04:26
3535098,A-4296014,2,2022-05-18 21:13:00,34.848189,-118.869107,4.525,Kern,CA,61.659699,64.838302,...,False,False,False,False,False,Day,2022,5,2,21:13:00
3535099,A-1453439,2,2020-05-30 14:49:06,34.184383,-118.910927,0.000,Ventura,CA,75.000000,46.000000,...,False,False,False,False,False,Day,2020,5,5,14:49:06


In [2]:
# Extract a random 10% of the data
model_df = model_df.sample(frac=0.1, random_state=78)

In [3]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 353510 entries, 3030768 to 30760
Data columns (total 33 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID                     353510 non-null  object 
 1   Severity               353510 non-null  int64  
 2   Start_Time             353510 non-null  object 
 3   Start_Lat              353510 non-null  float64
 4   Start_Lng              353510 non-null  float64
 5   Distance(mi)           353510 non-null  float64
 6   County                 353510 non-null  object 
 7   State                  353510 non-null  object 
 8   Temperature(F)         353510 non-null  float64
 9   Humidity(%)            353510 non-null  float64
 10  Pressure(in)           353510 non-null  float64
 11  Visibility(mi)         353510 non-null  float64
 12  Wind_Speed(mph)        353510 non-null  float64
 13  Precipitation(in)      353510 non-null  float64
 14  Weather_Condition      345746 n

In [4]:
# Drop the 'ID' and 'County' columns
model_df = model_df.drop(['ID', 'County'], axis=1)


In [5]:
# One-Hot Encode categorical variables
categorical_columns = ['State', 'Weather_Condition', 'Astronomical_Twilight']
model_df = pd.get_dummies(model_df, columns=categorical_columns)

In [6]:
# Convert 'Start_Time' to datetime
model_df['Start_Time'] = pd.to_datetime(model_df['Start_Time'])

# Add Hour to the dataset
model_df['Hour'] = model_df['Start_Time'].dt.hour

In [7]:
# Drop the 'Time' column 
model_df = model_df.drop('Time', axis=1)

# Drop the 'Start_Time' column as its components have been extracted
model_df = model_df.drop('Start_Time', axis=1)

# Check the dataframe info to confirm changes
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 353510 entries, 3030768 to 30760
Data columns (total 91 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Severity                           353510 non-null  int64  
 1   Start_Lat                          353510 non-null  float64
 2   Start_Lng                          353510 non-null  float64
 3   Distance(mi)                       353510 non-null  float64
 4   Temperature(F)                     353510 non-null  float64
 5   Humidity(%)                        353510 non-null  float64
 6   Pressure(in)                       353510 non-null  float64
 7   Visibility(mi)                     353510 non-null  float64
 8   Wind_Speed(mph)                    353510 non-null  float64
 9   Precipitation(in)                  353510 non-null  float64
 10  Amenity                            353510 non-null  bool   
 11  Bump                              

In [8]:
# Split data into features and target arrays
# Separate the features (X) and the target variable (y)
X = model_df.drop('Severity', axis=1)  # Features
y = model_df['Severity']  # Target variable

In [9]:
# Split data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=78)

In [10]:
# Scale data
scaler = StandardScaler()

# Fit the StandardScaler with the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [11]:
# need to find correlations between variables to focus on the onees that would have the biggest impact in the model and drop the ones that would not
# Compute the correlation matrix
correlation_matrix = model_df.corr()

# Convert the correlation matrix to a DataFrame
# Stack the correlation matrix and reset the index to turn it into a 'long' DataFrame
correlation_df = correlation_matrix.stack().reset_index()
correlation_df.columns = ['Feature1', 'Feature2', 'Correlation']

# Remove self-correlation entries (correlation of features with themselves)
correlation_df = correlation_df[correlation_df['Feature1'] != correlation_df['Feature2']]

# Sort the values by the absolute value of correlation to see the highest correlations
correlation_df['AbsCorrelation'] = correlation_df['Correlation'].abs()
correlation_df_sorted = correlation_df.sort_values('AbsCorrelation', ascending=False)

# Display the top 10 strongest correlations
print(correlation_df_sorted.head(10))

                         Feature1                     Feature2  Correlation  \
7918    Astronomical_Twilight_Day  Astronomical_Twilight_Night    -0.989551   
8007  Astronomical_Twilight_Night    Astronomical_Twilight_Day    -0.989551   
208                     Start_Lng                     State_CA    -0.762712   
2522                     State_CA                    Start_Lng    -0.762712   
1811              Traffic_Calming                         Bump     0.682960   
1010                         Bump              Traffic_Calming     0.682960   
123                     Start_Lat                     State_FL    -0.615289   
2971                     State_FL                    Start_Lat    -0.615289   
6827     Weather_Condition_Cloudy       Weather_Condition_Fair    -0.474260   
7005       Weather_Condition_Fair     Weather_Condition_Cloudy    -0.474260   

      AbsCorrelation  
7918        0.989551  
8007        0.989551  
208         0.762712  
2522        0.762712  
1811        0.6

In [12]:
#drop reduntant variable based on correlations analysis
model_df = model_df.drop('Astronomical_Twilight_Day', axis=1)


## Try 3 different models from simpler to more complex.  Will start with a Logistic Regression, then a Decision Tree, and finally a Deep Neural Net.

In [13]:
# Logistic Regression analysis
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Create a Logistic Regression model with multinomial option since we have more than two classes
lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

# Train the model using the training data
lr_model.fit(X_train_scaled, y_train)

# Predicting the target values for the test set
y_pred = lr_model.predict(X_test_scaled)

# Evaluating the model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.20      0.00      0.00       856
           2       0.82      0.97      0.89     70658
           3       0.48      0.17      0.25     14592
           4       0.22      0.00      0.01      2272

    accuracy                           0.80     88378
   macro avg       0.43      0.28      0.29     88378
weighted avg       0.74      0.80      0.75     88378

[[    1   854     1     0]
 [    4 68310  2329    15]
 [    0 12126  2449    17]
 [    0  1982   281     9]]


## add analysis for the Logistic Regression model

In [14]:
# Decision Tree model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Create a Decision Tree Classifier
dt_model = DecisionTreeClassifier(random_state=78)

# Train the model using the training data
dt_model.fit(X_train_scaled, y_train)

# Predicting the target values for the test set
y_pred = dt_model.predict(X_test_scaled)

# Evaluating the model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.43      0.42      0.43       856
           2       0.89      0.88      0.89     70658
           3       0.54      0.56      0.55     14592
           4       0.19      0.21      0.20      2272

    accuracy                           0.81     88378
   macro avg       0.51      0.52      0.52     88378
weighted avg       0.81      0.81      0.81     88378

[[  363   371   118     4]
 [  363 62361  6340  1594]
 [  106  5853  8201   432]
 [    9  1371   426   466]]


## add analysis for the decision tree model

In [15]:
# Deep Neural Model

# Convert labels to categorical one-hot encoding
y_one_hot = tf.keras.utils.to_categorical(y - 1)  # -1 because to_categorical assumes classes start at 0

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.25, random_state=78)

# Define the model
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 100
hidden_nodes_layer2 = 50

model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
model.add(tf.keras.layers.Dense(units=y_train.shape[1], activation='softmax'))

# Check the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               9100      
                                                                 
 dense_1 (Dense)             (None, 50)                5050      
                                                                 
 dense_2 (Dense)             (None, 4)                 204       
                                                                 
Total params: 14354 (56.07 KB)
Trainable params: 14354 (56.07 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [17]:
# Train the model
fit_model = model.fit(X_train_scaled, y_train, epochs=10, batch_size=50, validation_split=0.2)


Epoch 1/10
4243/4243 [==============================] - 20s 4ms/step - loss: 0.5033 - accuracy: 0.8063 - val_loss: 0.4779 - val_accuracy: 0.8109
Epoch 2/10
4243/4243 [==============================] - 19s 4ms/step - loss: 0.4605 - accuracy: 0.8159 - val_loss: 0.4568 - val_accuracy: 0.8155
Epoch 3/10
4243/4243 [==============================] - 19s 4ms/step - loss: 0.4430 - accuracy: 0.8202 - val_loss: 0.4439 - val_accuracy: 0.8191
Epoch 4/10
4243/4243 [==============================] - 19s 4ms/step - loss: 0.4325 - accuracy: 0.8219 - val_loss: 0.4425 - val_accuracy: 0.8190
Epoch 5/10
4243/4243 [==============================] - 18s 4ms/step - loss: 0.4252 - accuracy: 0.8234 - val_loss: 0.4354 - val_accuracy: 0.8213
Epoch 6/10
4243/4243 [==============================] - 20s 5ms/step - loss: 0.4192 - accuracy: 0.8254 - val_loss: 0.4361 - val_accuracy: 0.8194
Epoch 7/10
4243/4243 [==============================] - 18s 4ms/step - loss: 0.4150 - accuracy: 0.8272 - val_loss: 0.4254 - val_ac

In [18]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)

print(f"Test accuracy: {test_accuracy}")

2762/2762 - 7s - loss: 0.4273 - accuracy: 0.8258 - 7s/epoch - 2ms/step
Test accuracy: 0.8258163928985596


In [19]:

# Predict the classes for the test set
y_pred = model.predict(X_test_scaled)

# Convert the predictions from one-hot encoded vectors to the class labels
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert the true test set labels from one-hot encoded vectors to the class labels
y_true_classes = np.argmax(y_test, axis=1)

# Generate the classification report
clf_report = classification_report(y_true_classes, y_pred_classes)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

print(clf_report)
print(conf_matrix)

2762/2762 [==============================] - 8s 3ms/step
              precision    recall  f1-score   support

           0       0.51      0.39      0.44       856
           1       0.86      0.94      0.90     70658
           2       0.59      0.40      0.47     14592
           3       0.44      0.06      0.10      2272

    accuracy                           0.83     88378
   macro avg       0.60      0.45      0.48     88378
weighted avg       0.80      0.83      0.80     88378

[[  330   477    49     0]
 [  262 66763  3544    89]
 [   46  8708  5766    72]
 [    7  1692   448   125]]
